# Restart Analysis

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import rpy2.robjects as robjects
import multiprocessing
from experiments.pipelines.pipeline_rpy2 import Rpy2Pipeline

## Load analysis and project

In [3]:
project_id = 10
analysis_id = 38

In [4]:
project = Project.objects.get_or_create(id=project_id)[0]

DEBUG 2017-03-04 07:40:25,605 utils.py:91 | (0.006) SET SQL_AUTO_IS_NULL = 0; args=None
DEBUG 2017-03-04 07:40:25,607 utils.py:91 | (0.001) SELECT `projects_project`.`id`, `projects_project`.`title`, `projects_project`.`user_owner_id`, `projects_project`.`description`, `projects_project`.`created`, `projects_project`.`modified` FROM `projects_project` WHERE `projects_project`.`id` = 10 LIMIT 21; args=(10,)


In [5]:
analysis = Analysis.objects.get_or_create(id=analysis_id)[0]

DEBUG 2017-03-04 07:40:28,240 utils.py:91 | (0.001) SELECT `experiments_analysis`.`id`, `experiments_analysis`.`created`, `experiments_analysis`.`submited`, `experiments_analysis`.`owner`, `experiments_analysis`.`experiment_id`, `experiments_analysis`.`params_id`, `experiments_analysis`.`status` FROM `experiments_analysis` WHERE `experiments_analysis`.`id` = 38 LIMIT 21; args=(38,)


In [6]:
print analysis.status

Processing


In [7]:
analysis.status = 'Ready'
analysis.save()

DEBUG 2017-03-04 07:40:33,412 utils.py:91 | (0.001) UPDATE `experiments_analysis` SET `created` = '2017-02-28 20:27:40', `submited` = '2017-02-28 23:27:35', `owner` = 'admin', `experiment_id` = 17, `params_id` = 27, `status` = 'Ready' WHERE `experiments_analysis`.`id` = 38; args=(u'2017-02-28 20:27:40', u'2017-02-28 23:27:35', u'admin', 17, 27, 'Ready', 38)


In [8]:
databases_ids = Database.objects.all().exclude(name='standards').values_list('id', flat=True)
print databases_ids

DEBUG 2017-03-04 07:40:37,751 utils.py:91 | (0.002) SELECT `experiments_database`.`id` FROM `experiments_database` WHERE NOT (`experiments_database`.`name` = 'standards') LIMIT 21; args=('standards',)


[1L, 2L, 3L, 4L]


In [9]:
new_params = Params()
new_params.save()

for p in analysis.params.param.all():
    print p.id, p
    new_p = Parameter(state=p.state, name=p.name, value=p.value)
    new_p.save()
    new_params.param.add(new_p)    
    
for db in analysis.params.databases.all():
    print db.id, db
    new_params.databases.add(db.id)

DEBUG 2017-03-04 07:41:05,105 utils.py:91 | (0.000) INSERT INTO `experiments_params` (`id`) VALUES (DEFAULT); args=[]
DEBUG 2017-03-04 07:41:05,109 utils.py:91 | (0.001) SELECT `experiments_params`.`id` FROM `experiments_params` WHERE `experiments_params`.`id` = 27 LIMIT 21; args=(27,)
DEBUG 2017-03-04 07:41:05,115 utils.py:91 | (0.001) SELECT `experiments_parameter`.`id`, `experiments_parameter`.`name`, `experiments_parameter`.`value`, `experiments_parameter`.`state` FROM `experiments_parameter` INNER JOIN `experiments_params_param` ON ( `experiments_parameter`.`id` = `experiments_params_param`.`parameter_id` ) WHERE `experiments_params_param`.`params_id` = 27; args=(27,)
DEBUG 2017-03-04 07:41:05,119 utils.py:91 | (0.001) INSERT INTO `experiments_parameter` (`name`, `value`, `state`) VALUES ('iqr', '0.5000000000', 0); args=[u'iqr', u'0.5000000000', False]
DEBUG 2017-03-04 07:41:05,123 utils.py:91 | (0.001) SELECT `experiments_params_param`.`parameter_id` FROM `experiments_params_para

201 Parameter object
202 Parameter object
203 Parameter object
204 Parameter object
205 Parameter object
206 Parameter object
207 Parameter object
208 Parameter object
1 kegg
2 hmdb
3 lipidmaps
4 standard


### Creates a copy of an existing analysis

In [10]:
print analysis.id, analysis.experiment.id, analysis.params, analysis.owner, analysis.status

DEBUG 2017-03-04 07:43:19,423 utils.py:91 | (0.001) SELECT `experiments_experiment`.`id`, `experiments_experiment`.`title`, `experiments_experiment`.`created`, `experiments_experiment`.`modified` FROM `experiments_experiment` WHERE `experiments_experiment`.`id` = 17 LIMIT 21; args=(17,)


38 17 Params object admin Ready


In [11]:
print analysis.params

Params object


In [12]:
copy_analysis = Analysis.objects.create(
    owner = analysis.owner,
    experiment = analysis.experiment,
    params = new_params,
    status = 'Ready'
)

DEBUG 2017-03-04 07:43:23,717 utils.py:91 | (0.000) INSERT INTO `experiments_analysis` (`created`, `submited`, `owner`, `experiment_id`, `params_id`, `status`) VALUES ('2017-03-04 07:43:23', NULL, 'admin', 17, 28, 'Ready'); args=[u'2017-03-04 07:43:23', None, u'admin', 17, 28, 'Ready']


In [13]:
for p in copy_analysis.params.param.all():
    print p.id, p
    
for db in copy_analysis.params.databases.all():
    print db.id, db

DEBUG 2017-03-04 07:43:27,516 utils.py:91 | (0.001) SELECT `experiments_parameter`.`id`, `experiments_parameter`.`name`, `experiments_parameter`.`value`, `experiments_parameter`.`state` FROM `experiments_parameter` INNER JOIN `experiments_params_param` ON ( `experiments_parameter`.`id` = `experiments_params_param`.`parameter_id` ) WHERE `experiments_params_param`.`params_id` = 28; args=(28,)
DEBUG 2017-03-04 07:43:27,524 utils.py:91 | (0.001) SELECT `experiments_database`.`id`, `experiments_database`.`name` FROM `experiments_database` INNER JOIN `experiments_params_databases` ON ( `experiments_database`.`id` = `experiments_params_databases`.`database_id` ) WHERE `experiments_params_databases`.`params_id` = 28; args=(28,)


209 Parameter object
210 Parameter object
211 Parameter object
212 Parameter object
213 Parameter object
214 Parameter object
215 Parameter object
216 Parameter object
1 kegg
2 hmdb
3 lipidmaps
4 standard


In [14]:
print copy_analysis.id, copy_analysis.experiment.id, copy_analysis.params, copy_analysis.owner, copy_analysis.status

39 17 Params object admin Ready


In [15]:
pipeline = Rpy2Pipeline(copy_analysis, project)

INFO 2017-03-04 07:43:34,390 pipeline_rpy2.py:50 | ******************************************
INFO 2017-03-04 07:43:34,392 pipeline_rpy2.py:51 | Setup rpy2 connection
INFO 2017-03-04 07:43:34,393 pipeline_rpy2.py:52 | ******************************************


[1] "Initialising PuMP logging"
[1] "PuMP log level set to DEBUG"


/home/pimp/pimp/venv/local/lib/python2.7/site-packages/rpy2/robjects/packages.py:63: UserWarning: Note: the specification for S3 class "AsIs" in package 'RJSONIO' seems equivalent to one from package 'BiocGenerics': not turning on duplicate class definitions for this class.

  return _reval(expr)

WARNING 2017-03-04 07:43:41,807 packages.py:63 | /home/pimp/pimp/venv/local/lib/python2.7/site-packages/rpy2/robjects/packages.py:63: UserWarning: Note: the specification for S3 class "AsIs" in package 'RJSONIO' seems equivalent to one from package 'BiocGenerics': not turning on duplicate class definitions for this class.

  return _reval(expr)

DEBUG 2017-03-04 07:43:43,107 utils.py:91 | (0.003) SELECT `fileupload_picture`.`file` FROM `fileupload_picture` INNER JOIN `fileupload_samplefilegroup` ON ( `fileupload_picture`.`id` = `fileupload_samplefilegroup`.`posdata_id` ) INNER JOIN `fileupload_sample` ON ( `fileupload_samplefilegroup`.`id` = `fileupload_sample`.`samplefile_id` ) WHERE (`fileu

2017-03-04 07:43:41 ERROR:pump.misc.initialisePuMPLogging:Logging at ERROR level
2017-03-04 07:43:41 WARNING:pump.misc.initialisePuMPLogging:Logging at WARNING level
2017-03-04 07:43:41 INFO:pump.misc.initialisePuMPLogging:Logging at INFO level
2017-03-04 07:43:41 DEBUG:pump.misc.initialisePuMPLogging:Logging at DEBUG level
2017-03-04 07:43:42 INFO:PeakML.Isotope.plotSamples:OS type: x86_64-pc-linux-gnu
2017-03-04 07:43:42 INFO:PeakML.Isotope.plotSamples:Java parameters are: -Xmx8192m
2017-03-04 07:43:42 INFO:PeakML.Isotope.plotSamples:Using jar file /home/pimp/pimp/packrat/lib/x86_64-pc-linux-gnu/3.2.3/mzmatch.R/java/mzmatch-2.1-with-dependencies.jar
2017-03-04 07:43:42 INFO:PeakML.Isotope.plotSamples:Using SWT file /home/pimp/pimp/packrat/lib/x86_64-pc-linux-gnu/3.2.3/mzmatch.R/java/swt-4.5-gtk-linux-x86_64.jar
[1] "Initialising PiMP logging"
[1] "PiMP log level set to DEBUG"
2017-03-04 07:43:42 ERROR:Pimp.extra.initialisePiMPLogging:Logging at ERROR level
2017-03-04 07:43:42 WARNING

DEBUG 2017-03-04 07:43:43,140 utils.py:91 | (0.004) SELECT `fileupload_projfile`.`file` FROM `fileupload_calibrationsample` INNER JOIN `groups_projfileattribute` ON ( `fileupload_calibrationsample`.`id` = `groups_projfileattribute`.`calibrationsample_id` ) INNER JOIN `groups_attribute` ON ( `groups_projfileattribute`.`attribute_id` = `groups_attribute`.`id` ) LEFT OUTER JOIN `fileupload_standardfilegroup` ON ( `fileupload_calibrationsample`.`standardFile_id` = `fileupload_standardfilegroup`.`id` ) LEFT OUTER JOIN `fileupload_projfile` ON ( `fileupload_standardfilegroup`.`data_id` = `fileupload_projfile`.`id` ) WHERE (`fileupload_calibrationsample`.`project_id` = 10 AND `groups_attribute`.`name` = 'standard'); args=(10, 'standard')
DEBUG 2017-03-04 07:43:43,146 utils.py:91 | (0.001) SELECT `experiments_comparison`.`id`, `experiments_comparison`.`name`, `experiments_attributecomparison`.`attribute_id`, `groups_group`.`name`, `groups_attribute`.`name`, `experiments_attributecomparison`.`g

2017-03-04 07:43:43 INFO:Pimp.extra.setPiMPLoggerAnalysisID:Setting analysis ID in logger to 39
2017-03-04 07:43:43 39:INFO:Pimp.extra.setPiMPLoggerAnalysisID:Analysis id set


/home/pimp/pimp/venv/local/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Setting analysis directory to: analysis_39

  res = super(Function, self).__call__(*new_args, **new_kwargs)

WARNING 2017-03-04 07:43:43,871 functions.py:106 | /home/pimp/pimp/venv/local/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Setting analysis directory to: analysis_39

  res = super(Function, self).__call__(*new_args, **new_kwargs)



2017-03-04 07:43:43 39:INFO:Pimp.getAnalysisParams:Setting params
2017-03-04 07:43:43 39:DEBUG:Pimp.getAnalysisParams:Name: iqr Value: 0.5
2017-03-04 07:43:43 39:DEBUG:Pimp.getAnalysisParams:Name: rsd Value: 0.5
2017-03-04 07:43:43 39:DEBUG:Pimp.getAnalysisParams:Name: noise Value: 0.8
2017-03-04 07:43:43 39:DEBUG:Pimp.getAnalysisParams:Name: ppm Value: 3
2017-03-04 07:43:43 39:DEBUG:Pimp.getAnalysisParams:Name: mindetections Value: 3
2017-03-04 07:43:43 39:DEBUG:Pimp.getAnalysisParams:Name: minintensity Value: 5000
2017-03-04 07:43:43 39:DEBUG:Pimp.getAnalysisParams:Name: rtwindow Value: 0.05
2017-03-04 07:43:43 39:DEBUG:Pimp.getAnalysisParams:Name: rt.alignment Value: NA
2017-03-04 07:43:43 39:INFO:Pimp.extra.setPiMPLoggerAnalysisID:Setting analysis ID in logger to 39
2017-03-04 07:43:43 39:INFO:Pimp.extra.setPiMPLoggerAnalysisID:Analysis id set
2017-03-04 07:43:43 39:INFO:Pimp.createAnalysisDir:Setting working dir to /home/pimp/media/projects/10:
2017-03-04 07:43:43 39:INFO:Pimp.cre

In [16]:
xml_file_path = pipeline.run_pipeline()

INFO 2017-03-04 07:43:46,104 pipeline_rpy2.py:242 | Processing positive polarity
INFO 2017-03-04 07:43:46,107 pipeline_rpy2.py:619 | Creating /home/pimp/media/projects/10/analysis_39/positive
INFO 2017-03-04 07:43:46,112 pipeline_rpy2.py:619 | Creating /home/pimp/media/projects/10/analysis_39/positive/combined
INFO 2017-03-04 07:43:46,114 pipeline_rpy2.py:127 | ------------------------------------------
INFO 2017-03-04 07:43:46,115 pipeline_rpy2.py:128 | positive /home/pimp/media/projects/10/analysis_39/positive /home/pimp/media/projects/10/analysis_39/positive/combined
INFO 2017-03-04 07:43:46,117 pipeline_rpy2.py:129 | ------------------------------------------
DEBUG 2017-03-04 07:43:46,118 pipeline_rpy2.py:132 | Input for peak detection and RT correction [u'samples/POS/Beer_2_full1.mzXML', u'samples/POS/Beer_2_full2.mzXML', u'samples/POS/Beer_1_full1.mzXML', u'samples/POS/Beer_1_full2.mzXML']
/home/pimp/pimp/venv/local/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: User

Starting snow cluster with 8 local sockets.
Detecting features in file # 1 : Beer_2_full1.mzXML 
Detecting features in file # 2 : Beer_2_full2.mzXML 
Detecting features in file # 3 : Beer_1_full1.mzXML 
Detecting features in file # 4 : Beer_1_full2.mzXML 
center sample:  

DEBUG 2017-03-04 07:48:33,345 pipeline_rpy2.py:464 | Now creating /home/pimp/media/projects/10/analysis_39/positive/Beer_1_full1.peakml


Beer_1_full2 
Processing: Beer_2_full1  Beer_2_full2  Beer_1_full1  
2017-03-04 07:48:34 39:INFO:PeakML.xcms.write.SingleInstance:positive
2017-03-04 07:48:34 39:INFO:PeakML.xcms.write.SingleInstance:retrieving raw data


DEBUG 2017-03-04 07:48:57,457 pipeline_rpy2.py:464 | Now creating /home/pimp/media/projects/10/analysis_39/positive/Beer_1_full2.peakml
DEBUG 2017-03-04 07:49:28,610 pipeline_rpy2.py:464 | Now creating /home/pimp/media/projects/10/analysis_39/positive/Beer_2_full1.peakml


...

DEBUG 2017-03-04 08:07:42,857 utils.py:91 | (0.006) SELECT `groups_group`.`name`, `groups_attribute`.`name`, `fileupload_sample`.`name` FROM `groups_group` LEFT OUTER JOIN `groups_attribute` ON ( `groups_group`.`id` = `groups_attribute`.`group_id` ) LEFT OUTER JOIN `groups_sampleattribute` ON ( `groups_attribute`.`id` = `groups_sampleattribute`.`attribute_id` ) LEFT OUTER JOIN `fileupload_sample` ON ( `groups_sampleattribute`.`sample_id` = `fileupload_sample`.`id` ) WHERE `groups_group`.`id` IN (SELECT DISTINCT U0.`id` FROM `groups_group` U0 INNER JOIN `groups_attribute` U1 ON ( U0.`id` = U1.`group_id` ) INNER JOIN `experiments_attributecomparison` U2 ON ( U1.`id` = U2.`attribute_id` ) INNER JOIN `experiments_comparison` U3 ON ( U2.`comparison_id` = U3.`id` ) WHERE U3.`experiment_id` = 17); args=(17,)


2017-03-04 08:06:09 39:INFO:PuMP.callToArgList.dispatchCall:mzmatch.ipeak.util.Identify -i analysis_39/negative/final_combined_related.peakml -o analysis_39/negative/final_combined_related_identified_hmdb.peakml -ppm 3 -databases /home/pimp/pimp/packrat/lib/x86_64-pc-linux-gnu/3.2.3/PiMP/dbs/hmdb.xml -rtwindowrelative FALSE -adducts M-H,M-ACN+Na,M-Na,M-K,M-ACN+H
2017-03-04 08:06:49 39:INFO:PuMP.callToArgList.dispatchCall:mzmatch.ipeak.convert.ConvertToText -i analysis_39/negative/final_combined_related_identified_hmdb.peakml -o analysis_39/negative/mzmatch_output_hmdb.txt -annotations identification,ppm,adduct,relation.id,relation.ship,codadw,charge
2017-03-04 08:06:52 39:INFO:PuMP.callToArgList.dispatchCall:mzmatch.ipeak.util.Identify -i analysis_39/negative/final_combined_related.peakml -o analysis_39/negative/final_combined_related_identified_kegg.peakml -ppm 3 -databases /home/pimp/pimp/packrat/lib/x86_64-pc-linux-gnu/3.2.3/PiMP/dbs/kegg.xml -rtwindowrelative FALSE -adducts M-H,M-A

/home/pimp/pimp/venv/local/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: XML Reports

  res = super(Function, self).__call__(*new_args, **new_kwargs)

WARNING 2017-03-06 14:15:43,721 functions.py:106 | /home/pimp/pimp/venv/local/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: XML Reports

  res = super(Function, self).__call__(*new_args, **new_kwargs)

/home/pimp/pimp/venv/local/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Writing group information...

  res = super(Function, self).__call__(*new_args, **new_kwargs)

WARNING 2017-03-06 14:15:44,287 functions.py:106 | /home/pimp/pimp/venv/local/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Writing group information...

  res = super(Function, self).__call__(*new_args, **new_kwargs)

/home/pimp/pimp/venv/local/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Writing comparison information...

  res = super(Fun

2017-03-04 08:07:43 39:INFO:Pimp.extra.setPiMPLoggerAnalysisID:Analysis id set
2017-03-04 08:07:43 39:INFO:Pimp.runStats:Two polarities exist
2017-03-04 08:07:43 39:DEBUG:Pimp.runStats:Positive names: Mass,RT,Beer_1_full2,Beer_1_full1,Beer_2_full1,Beer_2_full2,relation.id,relation.ship,codadw,charge,hmdb_identification,hmdb_ppm,hmdb_adduct,kegg_identification,kegg_ppm,kegg_adduct,lipidmaps_identification,lipidmaps_ppm,lipidmaps_adduct,polarity
2017-03-04 08:07:43 39:DEBUG:Pimp.runStats:Negative names: Mass,RT,Beer_1_full2,Beer_1_full1,Beer_2_full1,Beer_2_full2,relation.id,relation.ship,codadw,charge,hmdb_identification,hmdb_ppm,hmdb_adduct,kegg_identification,kegg_ppm,kegg_adduct,lipidmaps_identification,lipidmaps_ppm,lipidmaps_adduct,polarity
2017-03-04 08:07:43 39:INFO:Pimp.runStats:isEqualNames: TRUE
2017-03-06 14:15:43 39:INFO:Pimp.runStats:Getting database connection
2017-03-06 14:15:43 39:INFO:Pimp.runStats:Saving Pimp.exportToXML.Robj_fixture
2017-03-06 14:15:44 39:INFO:Pimp.exp

/home/pimp/pimp/venv/local/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Writing pathway information...

  res = super(Function, self).__call__(*new_args, **new_kwargs)

WARNING 2017-03-06 14:20:37,227 functions.py:106 | /home/pimp/pimp/venv/local/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Writing pathway information...

  res = super(Function, self).__call__(*new_args, **new_kwargs)

DEBUG 2017-03-06 14:20:42,469 pipeline_rpy2.py:253 | xml_file_path is /home/pimp/media/projects/10/analysis_39/analysis_39.xml


### Delete analysis

In [74]:
from django.db.models.deletion import Collector

analysis_id = 26
analysis = Analysis.objects.get(id=analysis_id)
collector = Collector({})
collector.collect([analysis])
print analysis.id
print collector.dependencies

DEBUG 2017-02-27 17:11:21,063 utils.py:91 | (0.001) SELECT `experiments_analysis`.`id`, `experiments_analysis`.`created`, `experiments_analysis`.`submited`, `experiments_analysis`.`owner`, `experiments_analysis`.`experiment_id`, `experiments_analysis`.`params_id`, `experiments_analysis`.`status` FROM `experiments_analysis` WHERE `experiments_analysis`.`id` = 26 LIMIT 21; args=(26,)
DEBUG 2017-02-27 17:11:21,066 utils.py:91 | (0.001) SELECT `data_dataset`.`id`, `data_dataset`.`created`, `data_dataset`.`analysis_id` FROM `data_dataset` WHERE `data_dataset`.`analysis_id` IN (26); args=(26,)


26
{}


In [17]:
# deleting the params will cascade to the analysis and the dataset
analysis.params.delete()

DEBUG 2017-03-07 02:22:56,753 utils.py:91 | (0.000) SET SQL_AUTO_IS_NULL = 0; args=None
DEBUG 2017-03-07 02:22:56,768 utils.py:91 | (0.011) SELECT `experiments_analysis`.`id`, `experiments_analysis`.`created`, `experiments_analysis`.`submited`, `experiments_analysis`.`owner`, `experiments_analysis`.`experiment_id`, `experiments_analysis`.`params_id`, `experiments_analysis`.`status` FROM `experiments_analysis` WHERE `experiments_analysis`.`params_id` IN (27); args=(27,)
DEBUG 2017-03-07 02:22:56,892 utils.py:91 | (0.069) SELECT `data_dataset`.`id`, `data_dataset`.`created`, `data_dataset`.`analysis_id` FROM `data_dataset` WHERE `data_dataset`.`analysis_id` IN (38); args=(38,)
DEBUG 2017-03-07 02:22:56,913 utils.py:91 | (0.018) DELETE FROM `experiments_params_param` WHERE `experiments_params_param`.`params_id` IN (27); args=(27,)
DEBUG 2017-03-07 02:22:56,954 utils.py:91 | (0.039) DELETE FROM `experiments_params_databases` WHERE `experiments_params_databases`.`params_id` IN (27); args=(2

In [19]:
experiment = analysis.experiment
if len(experiment.analysis_set.all()) == 0:
    experiment.delete()

DEBUG 2017-03-07 02:23:56,356 utils.py:91 | (0.001) SELECT `experiments_analysis`.`id`, `experiments_analysis`.`created`, `experiments_analysis`.`submited`, `experiments_analysis`.`owner`, `experiments_analysis`.`experiment_id`, `experiments_analysis`.`params_id`, `experiments_analysis`.`status` FROM `experiments_analysis` WHERE `experiments_analysis`.`experiment_id` = 17; args=(17,)
